<a href="https://colab.research.google.com/github/agroimpacts/VegMapper/blob/dev-calval-simplify/calval/process_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a single training/validation/test set from multiple Collect Earth projects



### Table of Contents

* [Overview](#overview)
* [Set-up](#setup)
* [Sample preparation](#sample-prep)
    * [Read-in, reshape, and recode](#reshape-recode)
    * [Simplify the classes](#simplify)
    * [Calculate sample agreement](#agreement)
* [Split the dataset](#split)
    * [Combine and convert to spatial](#combine)

## Overview <a class="anchor" id="overview"></a>
This notebook demonstrates how several Collect Earth Online projects can be:

1. Provide functionality to check the structure and validity of user input for modeling; Yet, users are responsible for providing data in good format.
2. Re-code the class values and rename the column names.
2. Merged into a single dataset that provide a single label for each sample point and an estimate of label uncertainty;
3. Split into training, validation, and test (or map reference) samples.

The example data that can be used for this notebook, if you don't bring your own, are the results of three Collect Earth Online projects that were captured over the Department of Ucayali, Peru. They are in the VegMapper repo under the calval/data folder. Each project csv represents the efforts of an individual (or group of individuals working in the same project) to label 1350 points, classifying each into 1 of 4 classes: not oil palm; young oil palm; mature oil palm;  unsure. The datasets preserve all the information from these projects, although user email addresses were anonymized.


## Sample preparation <a class="anchor" id="sample-prep"></a>
Load packages, setup configuations, define a helper function...

In [ ]:
# @title (RUN) Install packages
%%capture
!pipe install folium

In [ ]:
# from label_utils import load_csv, subset_cols, rename_cols,\
#     check_exclusive, recode, combine_labelers, get_mode_and_occurence

#@title (RUN) Setup code
## Mount Drive
from google.colab import drive
root = '/content/gdrive'
drive.mount(root)

## Clone and/or update VegMapper
import os
# from datetime import datetime as dt
import pandas as pd
from sklearn.model_selection import train_test_split

repo_path = f"{root}/MyDrive/repos"
clone_path = 'https://github.com/agroimpacts/VegMapper.git'
if not os.path.exists(repo_path):
    print(f"Making {repo_path}")
    os.makedirs(repo_path, exist_ok=True)

if not os.path.exists(f"{repo_path}/VegMapper"):
    !git -C "{repo_path}" clone "{clone_path}"
else:
    !git -C "{repo_path}/VegMapper" pull

os.chdir(f"{repo_path}/VegMapper")

# Import sample_utils function
from vegmapper.calval.label_utils import *
from functools import partial
import folium

To load the CSV files, you only need to open the directory on the left panel of your Colab notebook. Then, navigate to the directory where you have the files, click on the three dots menue to the right of the file names, and select 'Copy Path.' Finally, paste the path in the box below.

In [ ]:
# @title (RUN) Load CEO Project CSVs

while True:
    try:
        num_users = int(input("Enter the number of CEO projects (The number of CEO projects must be more than 2): "))
        if num_users < 2:
            print("The number of CEO projects must be at least 2.")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter a valid number.")

# Initialize the list to store file paths
fs = []

# Loop to input file paths for each user
for i in range(num_users):
    while True:
        file_path = input(f"Enter the CSV file name for user {i+1}: ")
        if not file_path:
            print("File name cannot be empty.")
        else:
            fs.append(file_path)
            break

In [ ]:
# @title (RUN) Check that project "plot_id"s match
dataframes = []

# Read each CSV file into a DataFrame and store them in the list
for csv_file in fs:
    df = pd.read_csv(csv_file)
    dataframes.append(df)

# Check if 'plot_id' column values are the same in all DataFrames
plot_id_values = [df['plot_id'] for df in dataframes]

# Assuming all DataFrames have the same number of rows, you can use the `all`
# function
if all(plot_id.equals(plot_id_values[0]) for plot_id in plot_id_values):
    print("'plot_id' values are match in all CEO projects")
else:
    print("'plot_id' values must be same in all CEO projects")


**Note**: it is important to make sure that the CEO project files come from the same project, such each plot_id represents the same location. It is theoretically possible that 2 or more projects could have the same numbers of plots and plot_id numbers, but each plot_id represents a different location, in which case the results here will not be valid.

In [ ]:
# @title (RUN) Define and the class column names
df = pd.read_csv(fs[0])

print("\nColumn Names and Indices:")
for i, column in enumerate(df.columns):
    print(f"{i}: {column}")

new_col_names_input = input(
    "Enter the names you would like to represent the presence and absence \n"\
    "classes and any class you use to represent uncertainty, e.g. Presence,\n"\
    "Absence, Unsure. Separate each with a comma: "
)
new_col_names = [name.strip() for name in new_col_names_input.split(',')]
new_col_names

presence_columns_input = input(
    f"Enter column indices to change to '{new_col_names[0]}'"\
    " (separate with comma): "
)
presence_columns = [
    index.strip() for index in presence_columns_input.split(',')
]

absence_columns_input = input(
    f"Enter column indices to change to '{new_col_names[1]}'"\
    " (separate with comma): "
)
absence_columns = [index.strip() for index in absence_columns_input.split(',')]

not_sure_columns_input = input(
    f"Enter column indices to change to '{new_col_names[2]}'"\
     " (separate with comma): "
)
not_sure_columns = [
    index.strip() for index in not_sure_columns_input.split(',')
]

rename_dict = {
    "plot_id": "Point_ID",
    "pl_cluster": "Clust",
    "center_lat": "Lat",
    "center_lon": "Lon"
}
for i, column in enumerate(df.columns):
    if str(i) in presence_columns:
        rename_dict[column] = new_col_names[0]
    elif str(i) in absence_columns:
        rename_dict[column] = new_col_names[1]
    elif str(i) in not_sure_columns:
        rename_dict[column] = new_col_names[2]

print(rename_dict)
print(new_col_names)

In [ ]:
# @title Provide a numerical code presence (e.g. 1), absence (e.g. 0) and unsure (e.g. 2, if present)
recode_dict = {}

# Iterate through new_col_names and get user input for values
for column_name in new_col_names:
    arbitrary_number = int(input(f"Enter a number for the '{column_name}' category: "))
    recode_dict[column_name] = arbitrary_number

print("Updated recode_dict:")
print(recode_dict)

### Read-in, reshape, and recode classes <a class="anchor" id="reshape-recode"></a>
The first step was to combine the three datasets into a single dataset, with the columns from each of the three CEO projects, and to recode the four classes into a single column with values 0 (absence),  1 (presence), and 2 (unsure, assuming this category exists). At this step, there will be one column per CEO project. Each column contains the recoded classes (note the renaming of the columns is done in the next code chunk).

In [ ]:
# @title (RUN) Process and combine csvs
label_name = "labeler"

# Define a partial function with fixed arguments
process_csv_partial = partial(process_csv, rename_dict=rename_dict,
                              recode_dict=recode_dict,
                              new_col_names=new_col_names)

# Process ceo-survey-users one by one
dats = list(map(process_csv_partial, fs))

# Combine three datasets into one
combined = combine_labelers(dats, by=["Point_ID", "Clust"],
                            label_name=label_name, fs=fs)
combined_pl = combined.drop(columns=['Clust'])
combined_pl.head()

### Calculate sample agreement <a class="anchor" id="agreement"></a>
The next step was to calculate some agreement metrics across the three groups' samples. The primary approach was to calculate the proportion of labelling teams that selected the modal class. Since there were just three teams in this example, values were either 0.333, 0.667, 1. This agreement was calculated across for both the original classification scheme and the simplifed scheme, with columns `agree` and `agree2` providing the respective proportions for each observation.

In [ ]:
# @title (RUN) Agreement mode

num_labelers = len(fs)
labels = []

for i in range(1, num_labelers + 1):
    file_name = os.path.splitext(os.path.basename(fs[i-1]))[0]
    labels.append(file_name)

combined[['mode', 'mode_agreement']] = combined[labels].apply(get_mode_and_occurence, axis=1, result_type='expand')
pd.set_option('display.max_rows', None)
#print(combined)

# we can set the mode to -9999 if there is no agreement (mode_freq = 1/num_labelers)
combined.loc[combined['mode_agreement'] <= 1/3, 'mode'] = -9999

combined = combined.drop(combined[combined['mode'] == -9999].index)
print(f"Combined project has {combined.shape[0]} rows, {combined.shape[1]}"\
      "columns")

combined_pl2 = combined.drop(columns=['Clust'])

combined_pl2.head()

We can then calculate the average agreement per sample to get a sense of the uncertainty in labels for each class, for all 4 classes

In [ ]:
# @title (RUN) Calculate average agreement per sample
agreement = combined.groupby("mode").mean()
agreement = agreement.rename(columns={"mode_agreement": "mean agreement"})
print(agreement[['mean agreement']])

## Split the dataset <a class="anchor" id="split"></a>

Here we split the dataset into three parts for model training (60% of the sample), validation (20%), and final assessment (the 20% set aside as the test or map reference dataset).

The splits are confined to the usable sample, which is defined as samples not falling into class 3 and those with at least 2/3 observers agreeing on the class. This decision is made based on the simplified sample scheme (class2), rather than the full scheme (class). The resulting splits are denoted in a column called `usage` (this is distinct from the column `use`, which was used to filter out unusable observations).

Values of "unusable" in the `usage` column indicate observations that were not usable because of their low agreement or uncertain class.  They are included here for completeness, and in case they help with evaluation


In [ ]:
# @title (RUN) the splitting function
train_split = input("What proportion of the sample should be assigned"\
                    " to training?: ")
validation_split = input("What proportion of the sample should be assigned"\
                         " to validation?: ")
test_split = input("What proportion of the sample should be assigned"\
                   " to test/reference?: ")

# Check that the splits sum to 1
splits = np.array([train_split, validation_split, test_split]).astype(float)
split_string = f"The splits sum to {splits.sum()}, but must equal 1, "\
    "please try again"
assert splits.sum() == 1, split_string

#Set random seed for train/validation/reference split
seed = 999
train, test = train_test_split(combined, test_size=(1-splits[0]),
                                train_size=splits[0],
                                random_state=seed)

val, ref = train_test_split(test, test_size=splits[2],
                            train_size=splits[1],
                            random_state=seed)
out = pd.concat(
    [train.assign(usage="train"), val.assign(usage="validate"),
    ref.assign(usage="map_reference/test")]
).reset_index(drop=True)
out_pl = out.drop(columns=['Clust'])

out_pl.head()

### Combine and export to csv <a class="anchor" id="combine"></a>

The ineligible portion of the sample is also added back for completeness

In [ ]:
#@title (RUN) Export sample

gdrive_folder = input(f"Enter the name of the output folder: \n\n")
csv_name = input(f"Enter the name of the output csv file: \n\n")

output_dir = f"{root}/MyDrive/{gdrive_folder}"
os.makedirs(output_dir, exist_ok=True)

outpath = os.path.join(output_dir, csv_name)

with open(outpath, 'w') as f:
    out.to_csv(f, float_format='{:f}'.format, encoding='utf-8', index=False)

print('file exported')

And their locations on a map

In [ ]:
# @title (RUN) Display results

# For the usage category
color_mapping = {
    'train': 'blue',
    'validate': 'green',
    'map_reference/test': 'red',
    'unusable': 'gray'
}

m = folium.Map(location=[out['Lat'].mean(), out['Lon'].mean()], zoom_start=7)
scatter_group_usage = folium.FeatureGroup(name='Usage')

scatter_group_mode = folium.FeatureGroup(name='Mode')

# Create legend for 'usage'
legend_html_usage = '''
<div style="position: fixed; bottom: 50px; left: 50px; background-color: white; border: 2px solid grey; z-index: 9999; padding: 10px;">
    <h4>Usage</h4>
    <i style="background: blue; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Train<br>
    <i style="background: green; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Validate<br>
    <i style="background: red; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Map Reference/Test<br>
    <i style="background: gray; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Unusable<br>
</div>
'''
legend_usage = folium.Element(legend_html_usage)
m.get_root().html.add_child(legend_usage)

# Create legend for 'mode'
legend_html_mode = '''
<div style="position: fixed; bottom: 50px; left: 230px; background-color: white; border: 2px solid grey; z-index: 9999; padding: 10px;">
    <h4>Mode</h4>
    <i style="background: purple; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Absence<br>
    <i style="background: yellow; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Presence<br>
    <i style="background: black; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Not Sure<br>
</div>
'''
legend_mode = folium.Element(legend_html_mode)
m.get_root().html.add_child(legend_mode)

for usage, color in color_mapping.items():
    subset = out[out['usage'] == usage]
    for _, row in subset.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Lon']], radius=1, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(scatter_group_usage)

for mode in range(3):  # 0, 1, 2
    subset = out[out['mode'] == mode]
    color = ['purple', 'yellow', 'black'][mode]
    for _, row in subset.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Lon']], radius=1, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(scatter_group_mode)

# Add the feature groups to the map
scatter_group_usage.add_to(m)
scatter_group_mode.add_to(m)

# Add OpenStreetMap layer to the map
folium.TileLayer('openstreetmap').add_to(m)

# Add a layer control to toggle between 'usage' and 'mode' scatter plots and OpenStreetMap layer
folium.LayerControl(collapsed=False).add_to(m)

m